<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">

*このノートブックには、Jake VanderPlas による [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) からの抜粋が含まれています。コンテンツは利用可能です [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do).*

※テキストは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で、コードは[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode)で公開しています。このコンテンツが役立つと思われる場合は、[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode) による作業のサポートを検討してください!*

<!--ナビゲーション-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.03-Operations-in-Pandas.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>


# Pandas でのデータの操作

NumPy の重要な部分の 1 つは、基本的な算術演算 (加算、減算、乗算など) とより高度な演算 (三角関数、指数関数、対数関数など) の両方で、要素単位の演算をすばやく実行できることです。
Pandas はこの機能の多くを NumPy から継承しており、[Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) で導入した ufunc がその鍵となります。

ただし、Pandas にはいくつかの便利なひねりが含まれています。否定や三角関数などの単項演算の場合、これらの ufunc は出力で *インデックスと列ラベルを保持*し、加算や乗算などの 2 項演算の場合、Pandas は自動的に *インデックスを揃えます*オブジェクトを ufunc に渡します。
これは、データのコンテキストを維持し、さまざまなソースからのデータを結合すること (どちらも生の NumPy 配列を使用する潜在的にエラーが発生しやすいタスク) が、Pandas を使用すると本質的に誰にでもできるものになることを意味します。
さらに、1 次元の ``Series`` 構造と 2 次元の ``DataFrame`` 構造の間に明確に定義された操作があることを確認します。

## Ufuncs: インデックスの保存

Pandas は NumPy で動作するように設計されているため、NumPy ufunc は Pandas ``Series`` および ``DataFrame`` オブジェクトで動作します。
これを示す単純な ``Series`` と ``DataFrame`` を定義することから始めましょう:

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


これらのオブジェクトのいずれかに NumPy ufunc を適用すると、結果は別の Pandas オブジェクトになります *インデックスは保持されます:*

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

または、もう少し複雑な計算の場合:

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


[Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) で説明されている ufunc のいずれも、同様の方法で使用できます。

## UFuncs: インデックスの配置

2 つの ``Series`` または ``DataFrame`` オブジェクトに対するバイナリ操作の場合、Pandas は操作を実行する過程でインデックスを整列させます。
これは、以下のいくつかの例で説明するように、不完全なデータを扱う場合に非常に便利です。

### シリーズのインデックス配置

例として、2 つの異なるデータ ソースを組み合わせて、*面積* で上位 3 つの米国の州と *人口* で上位 3 つの米国の州のみを検索するとします。

In [6]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

これらを分割して人口密度を計算するとどうなるか見てみましょう。

In [7]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

結果の配列には、2 つの入力配列のインデックスの *union* が含まれます。これは、これらのインデックスに対して標準の Python 集合演算を使用して決定できます。

In [8]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

どちらか一方にエントリがないアイテムは、「NaN」または「非数」でマークされます。これは、Pandas が欠損データをマークする方法です (欠損データの詳細については、[Handling Missing Data](03.04-Missing-Values.ipynb) を参照してください)。
このインデックス マッチングは、Python の組み込み算術式のいずれに対してもこの方法で実装されます。欠損値はデフォルトで NaN で埋められます。

In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

NaN 値の使用が望ましくない場合は、演算子の代わりに適切なオブジェクト メソッドを使用して、塗りつぶし値を変更できます。
たとえば、 ``A.add(B)`` の呼び出しは ``A + B`` の呼び出しと同等ですが、 ``A`` または ``B`` の任意の要素のフィル値をオプションで明示的に指定できます。不足している可能性があります：

In [10]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### DataFrame のインデックス配置

``DataFrame`` で操作を実行する場合、列とインデックスの *両方* に対して同様のタイプの整列が行われます。

In [11]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [12]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [13]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


2 つのオブジェクトの順序に関係なく、インデックスが正しく配置され、結果のインデックスが並べ替えられることに注意してください。
``Series`` の場合と同様に、関連付けられたオブジェクトの算術メソッドを使用して、必要な ``fill_value`` を渡して、不足しているエントリの代わりに使用することができます。
ここでは、 ``A`` のすべての値の平均を入力します (最初に ``A`` の行を積み重ねることによって計算されます):

In [14]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


次の表に、Python 演算子と、それに相当する Pandas オブジェクト メソッドを示します。

| | Python オペレーター | Pandas メソッド |
|-----------------|---------------------------------------|
| | ``+`` | ``add()`` |
| | ``-`` | ``sub()``, ``subtract()`` |
| | ``*`` | ``mul()``, ``multiply()`` |
| | ``/`` | ``truediv()``, ``div()``, ``divide()``|
| | ``//`` | ``floordiv()`` |
| | ``%'' | ``mod()'' |
| | ``**`` | ``pow()`` |


## Ufuncs: DataFrame と Series 間の操作

``DataFrame`` と ``Series`` の間で操作を実行する場合、インデックスと列の配置は同様に維持されます。
``DataFrame`` と ``Series`` の間の操作は、2 次元と 1 次元の NumPy 配列の間の操作に似ています。
2 次元配列とその行の 1 つの違いを見つける一般的な操作を考えてみましょう。

In [15]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [16]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

NumPy のブロードキャスト ルール ([Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb) を参照) に従って、2 次元配列とその行の 1 つとの間の減算は、行単位で適用されます。

Pandas では、規則はデフォルトで同様に行単位で動作します。

In [17]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


代わりに列単位で操作したい場合は、前述のオブジェクト メソッドを使用して、 ``axis`` キーワードを指定します。

In [18]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


これらの ``DataFrame``/``Series`` 操作は、上記の操作と同様に、2 つの要素間のインデックスを自動的に整列させることに注意してください。

In [19]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int64

In [20]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


このインデックスと列の保持と整列は、Pandas のデータに対する操作が常にデータ コンテキストを維持することを意味します。これにより、生の NumPy 配列で異種のデータや整列されていないデータを操作するときに発生する可能性のある愚かなエラーの種類を防ぐことができます。

<!--ナビゲーション-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) >

<a href="https://colab.research.google.com/github/vitroid/PythonDataScienceHandbook/blob/ja/notebooks/03.03-Operations-in-Pandas.ipynb"><img align="left" src=" https://colab.research.google.com/assets/colab-badge.svg" alt="Colab で開く" title="Google Colaboratory で開いて実行する"></a>
